In [ ]:
# Run the pipeline on the image
# result = pipe("R.jpg")
result = pipe(final_data[1598]['image'])

# Print the result
print(result[0]['generated_text'])

In [ ]:
from transformers import TrainingArguments, Trainer, BlipProcessor, BlipForConditionalGeneration
from transformers import TFAutoModel
import os
import matplotlib.pyplot as plt  # Needed for plotting

# 1) Define training configuration
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/DEPI_Project/blip-finetuned",  # Directory to save model checkpoints
    per_device_train_batch_size=4,     # Batch size per GPU/CPU
    num_train_epochs=1,                # Total number of training epochs
    save_steps=125,                    # Save model every 125 steps
    save_total_limit=2,                # Keep only the last 2 checkpoints
    logging_dir='./logs',              # Directory for logs
    logging_steps=10,                  # Log training loss every 10 steps
    remove_unused_columns=False,       # Needed when using image inputs
    fp16=True,                         # Use 16-bit floating point precision if available
    run_name="blip-captioning",        # Name of the training run
    report_to="none",                  # Disable logging to external services (like WandB)
)

# 2) Initialize Trainer
trainer = Trainer(
    model=model,                # The BLIP model loaded earlier
    args=training_args,         # Training arguments defined above
    train_dataset=train_dataset, # The dataset to train on
)

# 3) Start training and save training metrics
train_result = trainer.train()
trainer.save_metrics("train", train_result.metrics)

# Plot training loss over steps
logs = trainer.state.log_history
steps = []
losses = []

for entry in logs:
    if "loss" in entry and "step" in entry:
        steps.append(entry["step"])
        losses.append(entry["loss"])

plt.figure(figsize=(8, 5))
plt.plot(steps, losses, marker='o')
plt.title("Training Loss over Steps")
plt.xlabel("Step")
plt.ylabel("Loss")
plt.grid(True)
plt.show()

# 4) Delete .safetensors files if they exist (to avoid conflicts)
model_path = "/content/drive/MyDrive/DEPI_Project/blip-finetuned"
for file in os.listdir(model_path):
    if file.endswith(".safetensors"):
        os.remove(os.path.join(model_path, file))

# 5) Save the model in PyTorch format
model.save_pretrained(model_path, safe_serialization=False)

# 6) Save the processor (tokenizer + image processor)
processor.save_pretrained(model_path)

# 7) Convert PyTorch model to TensorFlow format
tf_model = TFAutoModel.from_pretrained(
    model_path,    # Path to the saved PyTorch model
    from_pt=True   # Convert from PyTorch to TensorFlow
)
tf_model.save_pretrained(model_path, save_format="h5")

# 8) Confirm saved files
print(f"✅ Artifacts saved to {model_path}:")
for file in os.listdir(model_path):
    if file.endswith((".json", ".bin", ".h5", ".txt")):
        print(f"- {file}")
